In [ ]:
%matplotlib widget
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from scipy import constants
import pandas as pd

In [ ]:
def mktable(PCtoPG, PGtoPE, PEtoPC):
    data = pd.DataFrame(0, index=['PC', 'PG', 'PE'], columns=['PC', 'PG', 'PE'])
    data.loc['PC','PG'] = PCtoPG
    data.loc['PE','PC'] = PEtoPC
    data.loc['PG','PE'] = PGtoPE

    data.loc['PG','PC'] = -data.loc['PC','PG']
    data.loc['PC','PE'] = -data.loc['PE','PC']
    data.loc['PE','PG'] = -data.loc['PG','PE']
    
    return data

In [ ]:
def logSpace(xmin=-9, xmax=0, N=1000):
    grid_x = np.logspace(xmin, xmax, N)

    xPG, xPE = np.meshgrid(grid_x, grid_x) #titration grid

    xPC =  np.full((N, N), 1.0) - (xPG + xPE) #PC+PG+PE=1
    mask = xPC<0
    xPG=np.where(~mask, xPG, np.nan)
    xPE=np.where(~mask, xPE, np.nan)
    
    return xPC, xPG, xPE

In [ ]:
def linSpace(xmin=0, xmax=1, N=1000):
    grid_x = np.linspace(xmin, xmax, N)

    xPG, xPE = np.meshgrid(grid_x, grid_x) #titration grid

    xPC =  np.full((N, N), 1.0) - (xPG + xPE) #PC+PG+PE=1
    mask = xPC<0
    xPG=np.where(~mask, xPG, np.nan)
    xPE=np.where(~mask, xPE, np.nan)
    
    return xPC, xPG, xPE

In [ ]:
def genLogProb(E5, WT, RT):
    #Boltzmann weights
    KCG5  = np.exp(-E5.loc['PC','PG'] / RT)
    KCGWT = np.exp(-WT.loc['PC','PG'] / RT)
    KCE5  = np.exp(-E5.loc['PC','PE']/ RT)
    KCEWT = np.exp(-WT.loc['PC','PE'] / RT)

    data = (xPC + KCG5 * xPG + KCE5 * xPE) / (xPC + KCGWT * xPG + KCEWT * xPE)
    data = np.log(data)
    #data=np.where(mask, data, np.nan)
    
    return data

In [ ]:
##Old numbers
#DG_PG_5  = -7
#DG_PG_WT = -11
#DG_PE_5  = -2
#DG_PE_WT = -9


#Constants
temperature = 303.15 #Kelvin
RT = temperature*constants.R/(1000*constants.calorie) #kcal/mol

In [ ]:
#All data:
#Using a PG reference
WT_bin = mktable(PCtoPG=-7, PGtoPE=2, PEtoPC= 7-2)
E5_bin = mktable(PCtoPG=-4, PGtoPE=6, PEtoPC= 4-6)
WT_ter = mktable(PCtoPG=-6, PGtoPE=2, PEtoPC= 6-2)
E5_ter = mktable(PCtoPG=-3, PGtoPE=6, PEtoPC= 3-6)

In [ ]:
xPC, xPG, xPE = logSpace(-6, 0, 1000) #e^-9 to e^0, 1000 steps
#xPC, xPG, xPE = linSpace(0, 1, 1000) #e^-9 to e^0, 1000 steps
data = genLogProb(E5_ter, WT_ter, RT)

In [ ]:
def makeContourf(xPG, xPE, data, xmin=1e-9, xmax=1, log=True):
    %matplotlib widget
    
    fig, ax = plt.subplots() 
    cf = ax.contourf(xPG, xPE, data, levels=100)
    ax.scatter(0.25, 0.25, s=25, marker='p', c='k', linewidth=3)
    ax.scatter(0.25, xmin*1.2, s=25, marker='*', c='k', linewidth=3)
    ax.scatter(1/60, xmin*1.2, s=25, marker='P', c='k', linewidth=3)
    ax.scatter(xmin*1.2, 1/60, s=25, marker='X', c='k', linewidth=3)
    
    ax.scatter(0.25, 0.25, s=15, marker='p', label='2:1:1 PC:PG:PE')
    ax.scatter(0.25, xmin*1.2, s=15, marker='*', label='3:1 PC:PG')
    ax.scatter(1/60, xmin*1.2, s=15, marker='P', label='60:1 PC:PG')
    ax.scatter(xmin*1.2, 1/60, s=15, marker='X', label='60:1 PC:PE')
    
    ax.set_xlabel("PG content")
    ax.set_ylabel("PE content")
    #ax.title.set_text("Relative log-probability of ELIC5 conformation")
    ax.legend(loc='center right')
    
    if log:
        ax.set_xscale("log")
        ax.set_yscale("log") 
    
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([xmin,xmax])

    ax.set_aspect('equal')
    fig.colorbar(cf, ax=ax)
    
    return fig, ax

In [ ]:
#fig, ax = makeContourf(xPG, xPE, data)
fig, ax = makeContourf(xPG, xPE, data, xmin=1e-6, log=True)
plt.savefig('logloglog_pE5.png', dpi=600)

In [ ]:
%matplotlib widget
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(xPG, xPE, data)
ax.set_xlabel('PG')
ax.set_ylabel('PE')
ax.set_zlabel('log(E5) conformation')

ax.set_title('3D contour')
plt.show()


# Binding in a Ternary Mixture (implementation of derivation posted Sunday, April 3, 2022)

In [ ]:
def getRelProb(alpha, beta, xa, xb, stateTable, RT):
    prob = (xa/xb) * np.exp(-stateTable.loc[beta, alpha]/RT)
    return prob

In [ ]:
def getpAa(alpha, beta, gamma, xa, xb, xg, stateTable, RT):
    pA0a = 0
    pAba = getRelProb(beta, alpha, xb, xa, stateTable, RT)
    pAga = getRelProb(gamma, alpha, xg, xa, stateTable, RT)

    pAa = 1/(1+pAba+pAga+pA0a)
    
    return pAa

In [ ]:
#This is the same as pAa when pA0a is 0
def getfAa(alpha, beta, gamma, xa, xb, xg, stateTable, RT):
    pAa = getpAa(alpha, beta, gamma, xa, xb, xg, stateTable, RT)
    pAb = getpAa(beta, alpha, gamma, xb, xa, xg, stateTable, RT)
    pAg = getpAa(gamma, beta, alpha, xg, xb, xa, stateTable, RT)
    
    fAa = pAa/(pAa+pAb+pAg)
    
    return fAa

In [ ]:
#xPG = np.linspace(0, 1, 1000)
xPG = np.logspace(-6, 0, 100)

In [ ]:
xPX = 1-xPG
xPC = xPX*2/3
xPE = xPX/3
%matplotlib widget

#Binary mixture numbers currently make no appreciable difference
#fPC = getfAa('PC', 'PG', 'PE', xPC,xPG,xPE, WT_bin, RT)
#fPG = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_bin, RT)
#fPE = getfAa('PE', 'PG', 'PC', xPE,xPG,xPC, WT_bin, RT)

#plt.plot(xPG,fPC, label='fPC_bin')
#plt.plot(xPG,fPG, label='fPG_bin')
#lt.plot(xPG,fPE, label='fPE_bin')

fPC = getfAa('PC', 'PG', 'PE', xPC,xPG,xPE, WT_ter, RT)
fPG = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_ter, RT)
fPE = getfAa('PE', 'PG', 'PC', xPE,xPG,xPC, WT_ter, RT)

plt.plot(xPG,fPC, label='fPC_ter')
plt.plot(xPG,fPG, label='fPG_ter')
plt.plot(xPG,fPE, label='fPE_ter')

plt.xlabel('PG mol fraction')
plt.ylabel('Fraction of Sites Occupied')
plt.title('Wild Type PG Titration')

plt.legend()
plt.xscale('log')

plt.savefig('WT_PG_Titration_Ternary.png', dpi=600)

In [ ]:
xPX = 1-xPG
xPC = xPX*2/3
xPE = xPX/3

fPC = getfAa('PC', 'PG', 'PE', xPC,xPG,xPE, E5_ter, RT)
fPG = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, E5_ter, RT)
fPE = getfAa('PE', 'PG', 'PC', xPE,xPG,xPC, E5_ter, RT)

%matplotlib widget
plt.plot(xPG,fPC, label='fPC')
plt.plot(xPG,fPG, label='fPG')
plt.plot(xPG,fPE, label='fPE')

plt.legend()
plt.xscale('log')
plt.xlabel('PG mol fraction')
plt.ylabel('Fraction of Sites Occupied')
plt.title('ELIC5 PG Titration')

plt.savefig('ELIC5_PG_Titration_Ternary.png', dpi=600)

In [ ]:
xPX = 1-xPG
xPC = xPX*2/3
xPE = xPX/3
#fPC = []
#for x in np.arange(len(xPG)):
    #fPC.append(getpAa('PC', 'PG', 'PE', xPC[x],xPG[x],xPE[x], WT, RT))
#Or equivalently:

WTfPG_ter = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_ter, RT)
E5fPG_ter = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, E5_ter, RT)


xPC = 1-xPG
WTfPG_bin = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_bin, RT)
E5fPG_bin = getfAa('PG', 'PC', 'PE', xPG,xPC,xPE, E5_bin, RT)

%matplotlib widget
plt.plot(xPG,E5fPG_bin, label='E5 binary', linestyle='-', color='black')
plt.plot(xPG,WTfPG_bin, label='WT binary', linestyle='-', color='red')
plt.plot(xPG,E5fPG_ter, label='E5 ternary', linestyle='--', color='black')
plt.plot(xPG,WTfPG_ter, label='WT ternary', linestyle='--', color='red')


plt.legend()
plt.xscale('log')

plt.xlabel('PG mol fraction')
plt.ylabel('Fraction of Sites Occupied by PG')
plt.title('PG Titration Comparison')
#plt.xlim([1e-9, 1])
plt.savefig('PG_titration.png', dpi=600)

In [ ]:
#Generate numbers for barchart

xPC = 0.5
xPG = 0.25
xPE = 0.25

cols = ['fPC', 'fPG', 'fPE']
rows = pd.MultiIndex.from_tuples([('WT','2:1:1'), ('WT', '128:1'), ('E5', '2:1:1'), ('E5', '128:1')], names=['sequence', 'membrane'])
data = pd.DataFrame(columns=cols, index=rows)

data.loc[('WT','2:1:1'),'fPC'] = getpAa('PC', 'PG', 'PE', xPC,xPG,xPE, WT_ter, RT)
data.loc[('WT','2:1:1'),'fPG'] = getpAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_ter, RT)
data.loc[('WT','2:1:1'),'fPE'] = getpAa('PE', 'PG', 'PC', xPE,xPG,xPC, WT_ter, RT)

data.loc[('E5','2:1:1'),'fPC'] = getpAa('PC', 'PG', 'PE', xPC,xPG,xPE, E5_ter, RT)
data.loc[('E5','2:1:1'),'fPG'] = getpAa('PG', 'PC', 'PE', xPG,xPC,xPE, E5_ter, RT)
data.loc[('E5','2:1:1'),'fPE'] = getpAa('PE', 'PG', 'PC', xPE,xPG,xPC, E5_ter, RT)

xPC = 128/129
xPG = 1/129
xPE = 0
data.loc[('WT','128:1'),'fPC'] = getpAa('PC', 'PG', 'PE', xPC,xPG,xPE, WT_bin, RT)
data.loc[('WT','128:1'),'fPG'] = getpAa('PG', 'PC', 'PE', xPG,xPC,xPE, WT_bin, RT)

data.loc[('E5','128:1'),'fPC'] = getpAa('PC', 'PG', 'PE', xPC,xPG,xPE, E5_bin, RT)
data.loc[('E5','128:1'),'fPG'] = getpAa('PG', 'PC', 'PE', xPG,xPC,xPE, E5_bin, RT)


In [ ]:
%matplotlib widget

fig, (ax211, ax1281) = plt.subplots(1,2, sharey=True)
data.loc[(slice(None), '2:1:1'),:].plot.bar(ax=ax211, stacked=True, color={'fPC':'black', 'fPG':'grey', 'fPE':'red'})
ax211.set_xticklabels(['WT', 'E5'])
ax211.set_xlabel('2:1:1')

data.loc[(slice(None), '128:1'),:].plot.bar(ax=ax1281, stacked=True, color={'fPC':'black', 'fPG':'grey', 'fPE':'red'})
ax1281.set_xticklabels(['WT', 'E5'])
ax1281.set_xlabel('128:1') 
ax1281.get_legend().remove()

fig.suptitle('Absolute Occupancy Fraction')
ax211.set_ylabel('Occupancy Fraction')
#ax211.semilogy()
plt.savefig('OccupancyFraction.png', dpi=600)